In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.autograd import Variable
import os
import cv2
import numpy as np
from torchvision import datasets, models, transforms

In [4]:
vocab =  "<,.+:-?$ 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ>"
label_len = 36

In [5]:
# 각 vocab 사전을 만든다. 캐릭터와 토근간
char2token = {'PAD' : 0}
token2char = {0 : 'PAD'}
for i, c in enumerate(vocab):
    char2token[c] = i+1
    token2char[i+1] = c

In [8]:
# label 길이가 35개보다 크면, vocab에 label 한개한개가 포함되지 않으면 True
# 그 외는 False
def illegal(label):
    if len(label) > label_len-1:
        return True
    for l in label:
        if l not in vocab[1:-1]:
            return True
    return False

In [23]:
# 경로와 라벨 정보가 있는 파일을 로딩하고 한줄한줄 읽어
# 라벨str을 정수화함
class ListDataset(Dataset):
    def __init__(self, fname):
        self.lines = []
        # isinstance는 fname이 list형인지 확인 
        # if not, list형이 아니면 list로 만듬
        if not isinstance(fname, list):
            fname = [fname]
            
        # dataset file(fname)을 열어서 한줄 한줄 읽고 리스트화
        # readline 한줄씩 읽어서 \n은 버리고 \t로 split해서 [1]만 가져옴 label만 가져오는듯
        for f in fname:
            lines = open(f, encoding='UTF8').readlines()
            #lines = open(f, 'rt', encoding='UTF8').readlines()
            self.lines += [i for i in lines if not illegal(i.strip('\n').split('\t')[1])]
            
    def __len__(self):
        return len(self.lines)
    
    def __getitem__(self, index):
        line = self.lines[index]
        # 대략 line에는 "이미지 경로"와 "라벨"이 붙어있을 것으로 예상
        img_path, label_y_str = line.strip('\n').split('\t')
        img = cv2.imread(img_path) / 255.
        
        # Channels-first (채널 차원을 앞으로 뺌 : 파이토치여서)
        img = np.transpose(img, (2, 0, 1))
        
        # img은 numpy이기에 torch.from_numpy로 tensor로 변경
        img = torch.from_numpy(img).float()
        # label을 (36,) 만들어줌
        label = np.zeros(label_len, dtype=int)
        # label_str를 vocab사전에서 정수화
        for i, c in enumerate('<' + label_y_str):
            label[i] = char2token[c]
        label = torch.from_numpy(label)
        
        label_y = np.zeros(label_len, dtype=int)
        for i, c in enumerate(label_y_str + '>'):
            labe_y[i] = char2token[c]
        label_y = torch.from_numpy[label_y]

In [24]:
# (1,x,x)shape에서 상삼각행렬로 변환(주대각선성분 이하는 0)
# 그리고 0인값들은 True, 아니면 False로 표시
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    # np.triu(a, k=?) : 상삼각행렬에서 ?(주대각선성분에서 ?번째 위)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0

In [25]:
def make_std_mask(tgt, pad):
    "Create a mask to hide padding and future words."
    # -2번째에 차원 추가 예) 3,3,3 --> 3,3,1,3
    tgt_mask = (tgt != pad).unsqueeze(-2)
    tgt_mask = tgt_mask & Variable(subsequent_mask(tgt.size(-1)).
                                  type_as(tgt_mask.data))
    return tgt_mask

In [26]:
class Batch:
    "Object for holding a batch of data with mask during training."
    def __init__(self, imgs, trg_y, trg, pad=0):
        self.imgs = Variable(imgs.cuda(), requires_grad=False)
        self.src_mask = Variable(torch.from_numpy(np.ones([imgs.size(0), 1, 36], dtype=np.bool)).cuda())
        if trg is not None:
            self.trg = Variable(trg.cuda(), requires_grad=False)
            self.trg_y = Variable(trg_y.cuda(), requires_grad=False)
            self.trg_mask = self.mask_str_mask(self.trg, pad)
            self.ntokens = (self.trg_y != pad).data.sum()
    
    def make_std_mask(tgt, pad):
        "Create a mask to hide padding and future wodrds."
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & Variable(subsequent_mask(tgt.size(-1)).
                                      type_as(tgt_mask.data))
        return Variable(tgt_mask.cuda(), requires_grad=False)

In [27]:
class FeatureExtractor(nn.Module):
    def __init__(self, submodule, name):
        super(FeatureExtractor, self).__init__()
        self.submodule = submodule
        self.name = name
    def forward(self, x):
        # 아래 코드로 보아 submodule은 dictionary 형태임
        for name, module in self.submodule._modules.items():
            x = module(x)
            if name is self.name:
                b = x.size(0)
                c = x.size(1)
                # view는 차원을 바꾸고, permute는 차원 순서를 바꿈
                # (2,1,4) view(1,8), permute(4,1,2)
                return x.view(b, c, -1).permute(0, 2, 1)
        return None

In [28]:
if __name__=='__main__':
    listdataset = ListDataset('./data/test_dataset.txt')    # dataset이 있는 파일 경로 예)./data/test.txt
    dataloader = torch.utils.data.DataLoader(listdataset, batch_size=2, shuffle=False, num_workers=0)
    for epoch in range(1):
        for batch_i, (imgs, labels_y, labels) in enumerate(dataloader):
            continue

TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'